In [ ]:
import os
import sys
import subprocess
from tqdm import tqdm

# set path
nnpdf_path = os.path.expanduser("/home/daksh/nnpdf/nnpdf/nnpdf/n3fit/src/n3fit/scripts/")
validphys_script_path = os.path.expanduser("/home/daksh/anaconda3/envs/environment_nnpdf/lib/python3.12/site-packages/validphys/scripts/")
if nnpdf_path not in sys.path:
    sys.path.insert(0, nnpdf_path)
if validphys_script_path not in sys.path:
    sys.path.insert(0, validphys_script_path)

# import
try:
    from vp_setupfit import main as vp_setupfit
    from n3fit_exec import main as n3fit_exec
    from evolven3fit_exec import main as evolven3fit
    from postfit import main as postfit
    from vp_fitrename import main as vp_fitrename
    from vp_comparefits import main as vp_comparefits
except ImportError as e:
    print(f"Import failed: {e}")

runs = 2
version = '2.0'
runcards = ['bayesian', 'normal']
fitnames = [f'Bayesian_fit_{version}', f'Normal_fit_{version}']
num_replicas = 10
postfit_replicas = 1
compare_flag = True

# define a helper that runs the command and stays quiet
def run_cmd(cmd_list):
    # This runs the command in the background, 
    # sends logs to devnull, and waits for it to finish.
    subprocess.run(cmd_list, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

In [ ]:
for run in range(runs):
    runcard_name = f"Basic_runcard_{runcards[run]}"
    path_runcard_yml = f"/home/daksh/nnpdf/nnpdf/nnpdf/n3fit/runcards/examples/{runcard_name}.yml"

    # step 1: setup
    print("Initializing Setup...")
    run_cmd(["vp-setupfit", path_runcard_yml])
    print("Setup done")

    # step 2: fit
    print(f"Training {num_replicas} replicas...")
    for i in tqdm(range(num_replicas), desc="n3fit Progress"):
        run_cmd(["n3fit", path_runcard_yml, str(i+1)])
    print(f"Training finished")

    # step 3: evolve
    print("Running Evolution...")
    run_cmd(["evolven3fit", "evolve", runcard_name])    
    print("Evolution complete")

    # step 4: postfit
    print("Starting Postfit...")
    try:
        run_cmd(["postfit", str(postfit_replicas), runcard_name])
    except Exception as e:
        print(f"Postfit failed for {runcard_name}: {e}")
    print("Postfit completed")

In [ ]:
for run in range(runs+1):

    runcard_name = f"Basic_runcard_{runcards[run]}"
    # Step 5: Move to NNPDF results directory because vp-fitrename expects it there
    nnpdf_results_path = "/home/daksh/anaconda3/envs/environment_nnpdf/share/NNPDF/results"
    n3fit_path = "/home/daksh/anaconda3/envs/environment_nnpdf/lib/python3.12/site-packages/n3fit"

    print(f"Moving the {runcard_name} to NNPDF results directory...")
    result = subprocess.run(["cp", "-r", str(runcard_name), nnpdf_results_path], check=True)

    # Step 6: Rename and delete if the renamed version already exists from previous run
    save_name = fitnames[run]
    try:
        result = subprocess.run(["rm", "-r", nnpdf_results_path/save_name], check=True)
    except:
        pass

    print(f"Renaming {runcard_name} to {save_name}...")
    try:
        run_cmd(["vp-fitrename", runcard_name, save_name])
        print("done!!")
    except Exception as e:
        print(f"Rename failed: {e}")

    # Step 7: Move the renamed folder to results directory 
    #         and delete the original folder 
    result = subprocess.run(["cp", "-r", str(save_name), nnpdf_results_path], check=True)
    result = subprocess.run(["rm", "-r", f"{nnpdf_results_path}/{runcard_name}"])
    result = subprocess.run(["rm", "-r", f"{n3fit_path}/{save_name}"])



In [ ]:
if compare_flag:
    sys.argv = ["vp-comparefits", fitnames[0], fitnames[1], "--title", "Comparison Report", "--author", "Dakshansh", "--keywords", f"bayesian_report_{version}"]
    vp_comparefits()